In [1]:
import numpy as np
class GaussianDiscriminant:
    def __init__(self, k=2, d=8, priors=None, shared_cov=False):
        self.mean = np.zeros((k, d))  # mean
        self.shared_cov = shared_cov  # using class-independent covariance or not
        if self.shared_cov:
            self.S = np.zeros((d, d))  # class-independent covariance
        else:
            self.S = np.zeros((k, d, d))  # class-dependent covariance
        if priors is not None:
            self.p = priors
        else:
            self.p = [1.0 / k for i in range(k)]  # assume equal priors if not given
        self.k = k
        self.d = d

    def fit(self, Xtrain, ytrain):
        # compute the mean for each class
        C1,C2 = [],[]
        for i, j in enumerate(ytrain):
            if j == 1:
                C1.append(i)
            else:
                C2.append(i)
        self.mean[0] = Xtrain[C1].mean(axis=0)
        self.mean[1] = Xtrain[C2].mean(axis=0)

        if self.shared_cov:
            # compute the class-independent covariance
            self.S = np.cov(Xtrain, rowvar=0, ddof=0)
            pass  # placeholder
        else:
            # compute the class-dependent covariance
            for i in range(self.k):
                if i == 0:
                    self.S[i] = np.cov(Xtrain[C1], rowvar=0, ddof=0)
                else:
                    self.S[i] = np.cov(Xtrain[C2], rowvar=0, ddof=0)
            pass

    def predict(self, Xtest):
        # predict function to get predictions on test set
        predicted_class = np.ones(Xtest.shape[0])  # placeholder
        candidates_label = np.zeros((len(Xtest), 2))  # candidate label

        for i in np.arange(Xtest.shape[0]):  # for each test set example
            # calculate the value of discriminant function for each class
            for c in np.arange(self.k):
                if self.shared_cov:
                    log_P = np.log(self.p[c])
                    A = (
                        -0.5
                        *np.dot(
                            np.dot((Xtest[i]-self.mean[c]).T, np.linalg.inv(self.S)),
                            (Xtest[i]-self.mean[c]),
                        )
                    )
                    candidates_label[i][c] = log_P+A
                    pass  # placeholder
                else:
                    log_P = np.log(self.p[c])
                    log_S = -0.5*np.log(np.linalg.det(self.S[c]))
                    A = (
                        -0.5
                        *np.dot(
                            np.dot(
                                (Xtest[i]-self.mean[c]).T, np.linalg.inv(self.S[c])
                            ),
                            (Xtest[i]-self.mean[c]),
                        )
                    )

                    candidates_label[i][c] = log_P+log_S+A
                    pass

        predicted_class = np.argmax(candidates_label, axis=1)
        predicted_class += 1
        # determine the predicted class based on the values of discriminant function

        return predicted_class

    def params(self):
        if self.shared_cov:
            return self.mean[0], self.mean[1], self.S
        else:
            return self.mean[0], self.mean[1], self.S[0, :, :], self.S[1, :, :]



class GaussianDiscriminant_Diagonal:
    def __init__(self, k=2, d=8, priors=None):
        self.mean = np.zeros((k, d))  # mean
        self.S = np.zeros((d,))  # variance
        if priors is not None:
            self.p = priors
        else:
            self.p = [1.0 / k for i in range(k)]  # assume equal priors if not given
        self.k = k
        self.d = d

    def fit(self, Xtrain, ytrain):
        # compute the mean for each class
        C1,C2 = [],[]
        for i, j in enumerate(ytrain):
            if j == 1:
                C1.append(i)
            else:
                C2.append(i)
        self.mean[0] = Xtrain[C1].mean(axis=0)
        self.mean[1] = Xtrain[C2].mean(axis=0)
        # compute the variance of different features

        self.S = np.var(Xtrain, axis=0, ddof=0)

        pass  # placeholder

    def predict(self, Xtest):
        # predict function to get prediction for test set
        predicted_class = np.ones(Xtest.shape[0])  # placeholder
        candidates_label = np.zeros((len(Xtest), 2))  # candidate label

        for i in np.arange(Xtest.shape[0]):  # for each test set example
            # calculate the value of discriminant function for each class
            for c in np.arange(self.k):
                log_P = np.log(self.p[c])
                A = (
                    -0.5
                    *np.dot(
                        np.dot(
                            (Xtest[i]-self.mean[c]).T, np.linalg.inv(np.diag(self.S))
                        ),
                        (Xtest[i]-self.mean[c]),
                    )
                )
                candidates_label[i][c] = log_P+A
                pass

            # determine the predicted class based on the values of discriminant function
        predicted_class = np.argmax(candidates_label, axis=1)
        predicted_class += 1
        return predicted_class

    def params(self):
        return self.mean[0], self.mean[1], self.S

In [2]:
from MyDiscriminant import GaussianDiscriminant, GaussianDiscriminant_Diagonal
# load data
df = np.genfromtxt("training_data.txt", delimiter=",")
dftest = np.genfromtxt("test_data.txt", delimiter=",")
Xtrain = df[:, 0:8]
ytrain = df[:, 8]
Xtest = dftest[:, 0:8]
ytest = dftest[:, 8]

# define the model with a Gaussian Discriminant function (class-dependent covariance)
clf = GaussianDiscriminant(2, 8, [0.3, 0.7])

# update the model based on training data
clf.fit(Xtrain, ytrain)

# evaluate on test data
predictions = clf.predict(Xtest)
confusion_matrix = np.array(
    [
        [
            sum((ytest == 1) & (predictions == 1)),
            sum((ytest == 2) & (predictions == 1)),
        ],
        [
            sum((ytest == 1) & (predictions == 2)),
            sum((ytest == 2) & (predictions == 2)),
        ],
    ]
)
print("Confusion Matrix for Gaussian Discriminant with class-dependent covariance")
print(confusion_matrix)

# define the model with a Gaussian Discriminant function (class-independent covariance)
clf = GaussianDiscriminant(2, 8, [0.3, 0.7], shared_cov=True)

# update the model based on training data
clf.fit(Xtrain, ytrain)

# evaluate on test data
predictions = clf.predict(Xtest)
confusion_matrix = np.array(
    [
        [
            sum((ytest == 1) & (predictions == 1)),
            sum((ytest == 2) & (predictions == 1)),
        ],
        [
            sum((ytest == 1) & (predictions == 2)),
            sum((ytest == 2) & (predictions == 2)),
        ],
    ]
)
print("Confusion Matrix for Gaussian Discriminant with class-independent covariance")
print(confusion_matrix)

# define the model with a Gaussian Discriminant function (diagonal covariance)
clf = GaussianDiscriminant_Diagonal(2, 8, [0.3, 0.7])

# update the model based on training data
clf.fit(Xtrain, ytrain)

# evaluate on test data
predictions = clf.predict(Xtest)
confusion_matrix = np.array(
    [
        [
            sum((ytest == 1) & (predictions == 1)),
            sum((ytest == 2) & (predictions == 1)),
        ],
        [
            sum((ytest == 1) & (predictions == 2)),
            sum((ytest == 2) & (predictions == 2)),
        ],
    ]
)
print("Confusion Matrix for Gaussian Discriminant with diagonal covariance")
print(confusion_matrix)


Confusion Matrix for Gaussian Discriminant with class-dependent covariance
[[14  5]
 [16 65]]
Confusion Matrix for Gaussian Discriminant with class-independent covariance
[[24  5]
 [ 6 65]]
Confusion Matrix for Gaussian Discriminant with diagonal covariance
[[26 11]
 [ 4 59]]
